In [61]:
# Import libraries
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import haversine as hs
import gurobipy as gp
from gurobipy import GRB
from shapely import wkt

# Helper modules
import helper_population_allocation as pa
import helper_distance_calculation as dc

# Avoid printing set copy warnings
import warnings
warnings.filterwarnings("ignore")

### Set up dataset and parameters

In [62]:
# Get the main buildings dataset 
buildings_df = gpd.read_file('../processed_data/relevant_buildings.shp')

# Create ID variable
buildings_df.reset_index(drop=True, inplace=True)
buildings_df['building_id'] = buildings_df.index + 1
buildings_df['building_id'] = buildings_df.apply(lambda row: str(row['building_id']) + '-' + str(row['CLASS']) , axis=1)


In [63]:
# Get the proper geo IDs that can merge to USDA low income low access (from ACS data)
acs_data = pd.read_csv('ACSData.csv')
acs_data = acs_data[['geometry', 'geo_id', 'B01003_001E', 'B25010_001E']]
acs_data['geometry'] = acs_data['geometry'].apply(wkt.loads) # convert acs data to polygons
acs_data = gpd.GeoDataFrame(acs_data, geometry = 'geometry').set_crs(buildings_df.crs) # turn into polygons

# Do a spatial join from acs to buildings_df to get the propert geo_ids
buildings_df = buildings_df.sjoin(acs_data, how="left", predicate='intersects')
buildings_df['geo_id'] = buildings_df['geo_id'].str[-6:] # the last six characters is the tract ID'

# There are buildings that map onto multiple tracts based on the spatial join - don't know why this happens - see below
# buildings_df[buildings_df.duplicated(subset=['building_id'], keep=False)].sort_values('building_id')

# So we drop duplicates
buildings_df = buildings_df.drop_duplicates(subset=['building_id'], keep='first')


In [64]:
# Import usda low income low access data
usda_lila = pd.read_csv('../input_data/usda_lowincomelowaccess.csv')

usda_lila = usda_lila[['Allegheny_Tracts_GEOID','USDA_Data_LILATracts_1And10','USDA_Data_LILATracts_halfAnd10']]

# Get the right geo_id to merge
usda_lila['geo_id'] = usda_lila['Allegheny_Tracts_GEOID'].astype(str)


usda_lila['geo_id'] = usda_lila['geo_id'].str[-6:]

usda_lila

# Merge these definitions onto buildings_df
buildings_df = buildings_df.merge(usda_lila, how='left', on='geo_id')

# Drop building_id duplicates
buildings_df = buildings_df.drop_duplicates('building_id')

# If the buildings didn't merge, for now just assume LILA variables are 1
buildings_df[['USDA_Data_LILATracts_1And10','USDA_Data_LILATracts_halfAnd10']].fillna(1,inplace=True)

In [65]:
# How many tracts are LILA
tractlevel_access = buildings_df.groupby('geo_id').aggregate(lila_1_10 = ('USDA_Data_LILATracts_1And10', max))
tractlevel_access['lila_1_10'].value_counts()


0.0    125
1.0     10
Name: lila_1_10, dtype: int64

In [66]:
# # Filter out if not LILA at 1 mile
# buildings_df = buildings_df[buildings_df['USDA_Data_LILATracts_1And10'] == 1]

# # Keep only 1 census tract: hazelwood
buildings_df = buildings_df[buildings_df['geo_id'] == '562300']


In [67]:
# Create arrays to track ordering (residential)
res_buildings = buildings_df[buildings_df['class_reco'].str.contains('Residential')]
res_buildings = res_buildings.sort_values('building_id')
res_buildings = dc.get_geocoordinate(res_buildings, 'geometry')

res_buildings_array = np.array(res_buildings['building_id'])
res_buildings_coordinates_array = np.array(res_buildings['coordinates'])

# Create arrays to track ordering (Commercial)
comm_buildings = buildings_df[buildings_df['class_reco'].str.contains('commercial')]
comm_buildings = comm_buildings.sort_values('building_id')
comm_buildings = dc.get_geocoordinate(comm_buildings, 'geometry')

comm_buildings_array = np.array(comm_buildings['building_id'])
comm_buildings_coordinates_array = np.array(comm_buildings['coordinates'])

# # Create arrays to track ordering (grocery stores) - choosing one LILA tract, so no grocery stores
# grocery_stores = buildings_df[buildings_df['class_reco'].str.contains('Grocery')]
# grocery_stores = grocery_stores.sort_values('building_id')
# grocery_stores = dc.get_geocoordinate(grocery_stores, 'geometry')

# grocery_stores_array = np.array(grocery_stores['building_id'])
# grocery_stores_coordinates_array = np.array(grocery_stores['coordinates'])

In [68]:
print(f'There are now {len(comm_buildings_array)} commercial buildings')
print(f'There are now {len(res_buildings)} residential buildings')

There are now 560 commercial buildings
There are now 10306 residential buildings


In [69]:
# Import res comm access matrix again
#res_comm_distance_matrix = np.load('res_comm_distance_matrix.npy')

# Need to recalculate distance and access matrices based on new dataset (filtered)
res_comm_distance_matrix, res_comm_access_matrix = dc.calculate_access(res_buildings_coordinates_array, comm_buildings_coordinates_array)

# Creating a modified res comm access matrix 
res_comm_access_matrix = res_comm_distance_matrix.copy()
res_comm_access_matrix[res_comm_access_matrix <= 0.5] = 1
res_comm_access_matrix[res_comm_access_matrix != 1] = 0

# Create a modified res existing access array
# res_groc_distance_matrix, res_groc_access_matrix = dc.calculate_access(res_buildings_coordinates_array, grocery_stores_coordinates_array)

# res_groc_access_matrix = res_groc_distance_matrix.copy()
# res_groc_access_matrix[res_groc_access_matrix <= 0.5] = 1
# res_groc_access_matrix[res_groc_access_matrix != 1] = 0

# res_access_array = np.amax(res_groc_access_matrix, 1)

# Assume that none of the residential buildings have access 
res_access_array = np.zeros(len(res_buildings_array))

In [70]:
# Create parameter matrices (Res Population - Pj)
# ith value indicates the population in the ith column

# generate population numbers
res_buildings['population'] = 0 # baseline is nobody lives in a building
res_buildings.loc[res_buildings['class_reco'] == '1-Unit Residential', 'population'] = 1
res_buildings.loc[res_buildings['class_reco'] == '2-Unit Residential', 'population'] = 2
res_buildings.loc[res_buildings['class_reco'] == '3-Unit Residential', 'population'] = 3
res_buildings.loc[res_buildings['class_reco'] == '4+ Unit Residential', 'population'] = 4
# multiply by average household size
res_buildings['population'] = res_buildings['population']*res_buildings['B25010_001E'] 

res_population_array = np.array(res_buildings['population'])
res_population_array

array([1.94, 1.94, 1.94, ..., 3.68, 3.68, 3.68])

In [71]:
# Create demand matrix - number of unsatisfied customers
res_demand = res_population_array
res_demand[np.where(res_access_array == 1)] = 0 # setting effective demand to zero for buildings that have existing access to a grocery store

### Optimization

In [72]:
# Figure out thresholding later

# np.where(res_comm_distance_matrix <= 1)

# res_comm_distance_matrix <= 1


# pairings = {(c, r): res_comm_distance_matrix
#             for facility in range(num_candidates)
#             for cluster in range(num_clusters) 
#             if  dist(facility_locs[facility], centroids[cluster]) < threshold}
# print("Number of viable pairings: {0}".format(len(pairings.keys())))

In [73]:
### Testing modeling stuff

# Parameters
num_commercial_buildings = res_comm_distance_matrix.shape[1]
num_residential_buildings = len(res_demand)
num_stores = 1

# Implement model
m = gp.Model('Facility location')

# Decision variables 
select = m.addVars(range(num_commercial_buildings), vtype=GRB.BINARY, name='select') # select location
assign = m.addVars(range(num_residential_buildings), range(num_commercial_buildings), vtype=GRB.BINARY, name='assign') # assignment of residential building to cluster

# Objective function - min total distance from residential buildings to their assigned grocery store, multiplied by demand
m.setObjective(sum(sum(res_demand[i] * res_comm_distance_matrix[i,j] * assign[i, j] for i in range(num_residential_buildings)) for j in range(num_commercial_buildings)))
m.modelSense = GRB.MINIMIZE

# Constraints
m.addConstr(sum(select[i] for i in range(len(select))) <= num_stores, name='store_limit')

#m.addConstr(select.sum() <= num_stores, name = 'store_limit') # Facility limit
for i in range(num_residential_buildings):
    m.addConstr(sum(assign[i,j] for j in range(num_commercial_buildings)) ==  1) # can only assign each residential building to one store

for i in range(num_residential_buildings):
    for j in range(num_commercial_buildings):
        m.addConstr(assign[i,j] <= select[j], name='open2assign') # locations can only be assigned demand if they are selected

# Optimize
m.optimize()


Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 5781667 rows, 5771920 columns and 17314640 nonzeros
Model fingerprint: 0x23dafd21
Variable types: 0 continuous, 5771920 integer (5771920 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-03, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 0 rows and 0 columns (presolve time = 5s) ...
Presolve removed 0 rows and 0 columns (presolve time = 53s) ...
Presolve removed 0 rows and 0 columns (presolve time = 55s) ...
Presolve removed 0 rows and 0 columns (presolve time = 60s) ...
Presolve removed 0 rows and 0 columns (presolve time = 65s) ...
Presolve removed 0 rows and 0 columns (presolve time = 70s) ...
Presolve removed 0 rows and 0 columns (presolve time = 75s) ...


### Results Analysis

In [ ]:
m.objVal

725.4090693131119

In [ ]:
counter = 0
for i in range(len(select)):
    counter += select[i].x

counter

1.0